In [4]:
import splitfolders, os

# C:\ProjectWork\Basic_Python\Zero_shot_Learning\Dataset\CUB_KMean_Dataset\ViT_data
row_dataset_path = os.path.join('..', 'Dataset', 'CUB_KMean_Dataset', 'images')
split_dataset_path = os.path.join('..', 'Dataset', 'data', 'CUB_data', 'images')

# splitfolders.ratio(row_dataset_path, split_dataset_path, seed=1111, ratio=(0.7, 0.15, 0.15))

# Libraries

In [11]:
# libs
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout


# variables

In [12]:
# variables

# Set dataset paths
train_dir = os.path.join(split_dataset_path, 'train')
val_dir = os.path.join(split_dataset_path, 'val') 
test_dir = os.path.join(split_dataset_path, 'test') 
num_classes = len(os.listdir(train_dir))

# Image dimensions
img_size = (224, 224)  # Resize to 128x128
batch_size = 16
latent_dim = 16  # Dimension of latent space



In [13]:

# # Load dataset
# datagen = ImageDataGenerator(rescale=1./255)

# train_generator = datagen.flow_from_directory(train_dir, 
#                                               target_size=img_size, 
#                                               batch_size=batch_size)

# val_generator = datagen.flow_from_directory(val_dir, 
#                                             target_size=img_size, 
#                                             batch_size=batch_size)

# test_generator = datagen.flow_from_directory(test_dir, 
#                                              target_size=img_size, 
#                                              batch_size=batch_size)


# Autoencoder

In [14]:
def build_autoencoder(input_shape):
    # Encoder
    encoder = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same')
    ])

    # Decoder
    decoder = models.Sequential([
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # Assuming 3-channel images
    ])

    # Autoencoder
    autoencoder = models.Sequential([encoder, decoder])
    return autoencoder, encoder

# Prepare Data

In [15]:
# Set dataset paths
train_dir = os.path.join(split_dataset_path, 'train')
val_dir = os.path.join(split_dataset_path, 'val') 
test_dir = os.path.join(split_dataset_path, 'test') 
num_classes = len(os.listdir(train_dir))

# Image dimensions
img_height, img_width = 224, 224
batch_size = 16

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input'  # For autoencoder, we use 'input' as target is the same as input
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input'
)

Found 4146 images belonging to 101 classes.
Found 872 images belonging to 101 classes.
Found 921 images belonging to 101 classes.


# Train Autoencoder

In [20]:
input_shape = (img_height, img_width, 3)
autoencoder, encoder = build_autoencoder(input_shape)

autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the autoencoder
autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=val_generator.samples // batch_size
)

Epoch 1/5
259/259 [==============================] - 404s 2s/step - loss: 0.0145 - accuracy: 0.6567 - val_loss: 0.0079 - val_accuracy: 0.7962
Epoch 2/5
259/259 [==============================] - 399s 2s/step - loss: 0.0073 - accuracy: 0.8059 - val_loss: 0.0069 - val_accuracy: 0.8176
Epoch 3/5
259/259 [==============================] - 400s 2s/step - loss: 0.0064 - accuracy: 0.8205 - val_loss: 0.0064 - val_accuracy: 0.8269
Epoch 4/5
259/259 [==============================] - 399s 2s/step - loss: 0.0061 - accuracy: 0.8188 - val_loss: 0.0061 - val_accuracy: 0.8031
Epoch 5/5
259/259 [==============================] - 399s 2s/step - loss: 0.0058 - accuracy: 0.8243 - val_loss: 0.0062 - val_accuracy: 0.8194


# Extract Features Using Encoder

In [21]:
def extract_features(generator, encoder):
    features = []
    labels = []
    for i in range(len(generator)):
        x, _ = generator[i]  # Discard the target (input) for feature extraction
        encoded_features = encoder.predict(x)
        features.append(encoded_features)
        labels.append(generator.classes[generator.index_array[i*batch_size:(i+1)*batch_size]])
    return np.vstack(features), np.hstack(labels)

train_features, train_labels = extract_features(train_generator, encoder)
val_features, val_labels = extract_features(val_generator, encoder)
test_features, test_labels = extract_features(test_generator, encoder)

1/1 [==============================] - 0s 143ms/step


1/1 [==============================] - 0s 141ms/step


1/1 [==============================] - 0s 113ms/step


# Train Classifier

In [22]:
num_classes = 101

# Define classifier
classifier = models.Sequential([
    layers.Input(shape=train_features.shape[1:]),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train classifier
classifier.fit(
    train_features, train_labels,
    validation_data=(val_features, val_labels),
    epochs=30,
    batch_size=batch_size
)

Epoch 1/30
260/260 [==============================] - 90s 334ms/step - loss: 4.7483 - accuracy: 0.0082 - val_loss: 4.5978 - val_accuracy: 0.0172
Epoch 2/30
260/260 [==============================] - 84s 324ms/step - loss: 4.6071 - accuracy: 0.0101 - val_loss: 4.5878 - val_accuracy: 0.0161
Epoch 3/30
260/260 [==============================] - 83s 321ms/step - loss: 4.5995 - accuracy: 0.0099 - val_loss: 4.5664 - val_accuracy: 0.0149
Epoch 4/30
260/260 [==============================] - 84s 322ms/step - loss: 4.5723 - accuracy: 0.0099 - val_loss: 4.5433 - val_accuracy: 0.0161
Epoch 5/30
260/260 [==============================] - 84s 322ms/step - loss: 4.5600 - accuracy: 0.0104 - val_loss: 4.5240 - val_accuracy: 0.0172
Epoch 6/30
260/260 [==============================] - 84s 321ms/step - loss: 4.5487 - accuracy: 0.0109 - val_loss: 4.5425 - val_accuracy: 0.0183
Epoch 7/30
260/260 [==============================] - 83s 321ms/step - loss: 4.5705 - accuracy: 0.0116 - val_loss: 4.5233 - val_ac

# Save the Model

In [23]:
classifier.save('classification_model.h5')
encoder.save('encoder_model.h5')

C:\ProjectWork\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Single Image Classification

In [24]:
from tensorflow.keras.preprocessing import image

def classify_single_image(image_path, encoder, classifier):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Extract features using encoder
    encoded_features = encoder.predict(img_array)

    # Predict class
    predictions = classifier.predict(encoded_features)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    return predicted_class

# Example usage
image_path = os.path.join(train_dir, 'Acadian_Flycatcher', 'Acadian_Flycatcher_0003_29094.jpg')
predicted_class = classify_single_image(image_path, encoder, classifier)
print(f"Predicted Class: {predicted_class}")

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir)
class_names = train_dataset.class_names
predicted_class_name = class_names[predicted_class]
print(f"Predicted Class Name: {predicted_class_name}")

1/1 [==============================] - 0s 212ms/step
Predicted Class: 63
Found 4146 files belonging to 101 classes.
Predicted Class Name: Henslow_Sparrow
